In [109]:
#impoort dependencies
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sklearn.linear_model import LinearRegression
from sqlalchemy.ext.declarative import declarative_base
from pprint import pprint
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [110]:
#Importing the data tabl;e
All_data_df = pd.read_csv("./Clean_datasets/mergeFinalFilledwithedu.csv")
All_data_df.head()

,LOCATION,Year,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
0,Australia,1960,2412.627589,70.610,10275000.0,1.3,3.45,20.1,NaN
1,Australia,1961,2383.188902,70.700,10508200.0,1.4,3.54,19.8,NaN
2,Australia,1962,2577.332834,70.790,10700500.0,1.4,3.43,20.3,NaN
3,Australia,1963,2752.620592,70.880,10906900.0,1.4,3.34,19.5,NaN
4,Australia,1964,2902.590472,70.872,11121600.0,1.4,3.15,18.7,NaN


In [111]:
#cleaning data by removing all NaN
data_nonulls_df=All_data_df.dropna()
data_nonulls_df.describe()

,Year,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
count,412.000000,412.000000,412.000000,4.120000e+02,412.000000,412.000000,412.000000,412.000000
mean,2008.097087,28182.057263,77.276762,1.240653e+08,140.374029,1.770777,9.291748,33.317541
std,9.452933,15569.616894,4.937650,1.810933e+08,143.561538,0.403363,8.971921,17.053884
min,1981.000000,1136.755200,57.200000,1.681442e+07,2.200000,0.880000,1.800000,5.596642
25%,2001.000000,15389.205930,74.986000,5.132588e+07,25.175000,1.440000,3.700000,19.082791
50%,2009.500000,27373.057350,78.123000,6.647405e+07,98.750000,1.755000,5.400000,30.553422
75%,2016.000000,40024.395962,81.029000,1.276360e+08,233.450000,2.000000,11.075000,47.778956
max,2022.000000,76359.999520,84.910000,1.417173e+09,524.000000,3.050000,53.700000,69.851555


In [112]:

countries=data_nonulls_df["LOCATION"].unique()
print(countries)

['Australia' 'Canada' 'France' 'Germany' 'Italy' 'Japan'
 'Republic of Korea' 'Mexico' 'Türkiye' 'United Kingdom'
 'United States of America' 'Brazil' 'Russian Federation' 'China' 'India'
 'Indonesia' 'South Africa' 'Argentina' 'Saudi Arabia']


In [113]:
#cleaning data by removing all columns that are considered not needed
#,"G20 Value"
data_training=data_nonulls_df.drop(columns=["LOCATION","Year"])
data_training

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
29,17835.12296,76.490,16814416.0,2.2,1.84,8.5,34.140335
31,18177.56368,77.090,17284036.0,2.2,1.86,6.9,35.707539
33,20184.94601,77.690,17634808.0,2.3,1.87,6.0,22.836517
34,21432.77545,77.920,17805468.0,2.3,1.85,6.0,23.624947
35,22442.31684,78.150,18004882.0,2.3,1.83,5.7,24.697142
...,...,...,...,...,...,...,...
2641,11795.16290,76.450,44494502.0,15.9,1.62,2.9,17.455540
2642,9963.67420,76.594,44938712.0,16.0,1.61,3.2,18.856657
2643,8496.42820,76.738,45376763.0,16.1,1.58,3.2,18.214817
2644,10636.11550,76.882,45808747.0,16.2,1.58,3.1,19.026117


In [114]:

fertility_y = data_training['TotalFertilityRate(LiveBirthsPerWoman)']
X=data_training.drop(columns="TotalFertilityRate(LiveBirthsPerWoman)")
#X=data_training.copy()


model = LinearRegression()
model.fit(X, fertility_y)
coef=model.coef_
intercept=model.intercept_
print(f"Model's slope: {model.coef_}")


Model's slope: [ 1.94522504e-06  6.95436870e-03  4.74217584e-11 -1.03128190e-03
  3.94456543e-02  1.16474514e-03]


In [115]:
predicted_y_values = model.predict(X)
predicted_y_values

array([1.85231584, 1.79588921, 1.75338759, 1.75834082, 1.75132871,
       1.75168461, 1.73880328, 1.77227556, 1.75566939, 1.76265877,
       1.76592601, 1.76372399, 1.7643243 , 1.7871748 , 1.78209313,
       1.76961711, 1.77345796, 1.77305438, 1.77594557, 1.76879014,
       1.75217183, 1.77106223, 1.76672976, 1.75573141, 1.75950766,
       1.776763  , 1.77377345, 1.77167281, 1.77707126, 1.7899407 ,
       1.80827479, 1.87211593, 1.79454774, 1.77674036, 1.76578056,
       1.77486371, 1.77990006, 1.77721352, 1.7618435 , 1.76249141,
       1.75567824, 1.76141348, 1.76437147, 1.77580724, 1.78305529,
       1.79023933, 1.79529363, 1.81351074, 1.81217558, 1.82693282,
       1.82831472, 1.81071524, 1.82956907, 1.83630733, 1.83544301,
       1.84130521, 1.83090655, 1.81100917, 1.81066285, 1.81714433,
       1.82534299, 1.81958111, 1.81235447, 1.83321416, 1.84006291,
       1.75934545, 1.69357134, 1.69046781, 1.67496304, 1.65111275,
       1.61573462, 1.61578544, 1.61729476, 1.60990836, 1.62278

In [116]:

predicted=data_training.copy()
predicted["fertility predicted"]=predicted_y_values
predicted.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value,fertility predicted
29,17835.12296,76.49,16814416.0,2.2,1.84,8.5,34.140335,1.852316
31,18177.56368,77.09,17284036.0,2.2,1.86,6.9,35.707539,1.795889
33,20184.94601,77.69,17634808.0,2.3,1.87,6.0,22.836517,1.753388
34,21432.77545,77.92,17805468.0,2.3,1.85,6.0,23.624947,1.758341
35,22442.31684,78.15,18004882.0,2.3,1.83,5.7,24.697142,1.751329


In [117]:
from sklearn.metrics import mean_squared_error, r2_score
score = model.score(X, fertility_y, sample_weight=None)
r2 = r2_score(fertility_y, predicted_y_values)
mse = mean_squared_error(fertility_y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(fertility_y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.7597335175224442.
The r2 is 0.7597335175224442.
The mean squared error is 0.03899685467583898.
The root mean squared error is 0.19747621293674583.
The standard deviation is 0.40287302998837765.


# Regression without null values, normalized


In [119]:
data_training_norm=data_training.copy()

columns=[]
column_names=data_training_norm.columns.values.tolist()
columns.append(column_names)

data_training_norm = StandardScaler().fit_transform(data_training_norm[column_names])
data_training_norm=pd.DataFrame(data_training_norm, columns=column_names,index=data_training.index)

data_training_norm.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
29,-0.665367,-0.159533,-0.592961,-0.963643,0.171824,-0.088355,0.048305
31,-0.643346,-0.037870,-0.590365,-0.963643,0.221468,-0.266906,0.140314
33,-0.514260,0.083793,-0.588425,-0.962945,0.246289,-0.367340,-0.615330
34,-0.434018,0.130430,-0.587482,-0.962945,0.196646,-0.367340,-0.569042
35,-0.369098,0.177068,-0.586379,-0.962945,0.147002,-0.400819,-0.506095


In [120]:
fertility_y = data_training_norm['TotalFertilityRate(LiveBirthsPerWoman)']
X=data_training_norm.drop(columns="TotalFertilityRate(LiveBirthsPerWoman)")
#X=data_training.copy()


model = LinearRegression()
model.fit(X, fertility_y)
coef=model.coef_
intercept=model.intercept_
predicted_y_values = model.predict(X)
predicted=data_training4.copy()
predicted["fertility predicted"]=predicted_y_values
predicted.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value,fertility predicted
29,-0.665367,-0.159533,-0.592961,-0.963643,0.171824,-0.088355,0.048305,0.202394
31,-0.643346,-0.037870,-0.590365,-0.963643,0.221468,-0.266906,0.140314,0.062334
33,-0.514260,0.083793,-0.588425,-0.962945,0.246289,-0.367340,-0.615330,-0.043163
34,-0.434018,0.130430,-0.587482,-0.962945,0.196646,-0.367340,-0.569042,-0.030868
35,-0.369098,0.177068,-0.586379,-0.962945,0.147002,-0.400819,-0.506095,-0.048273


In [121]:
from sklearn.metrics import mean_squared_error, r2_score
score = model.score(X, fertility_y, sample_weight=None)
r2 = r2_score(fertility_y, predicted_y_values)
mse = mean_squared_error(fertility_y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(fertility_y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.7597335175224443.
The r2 is 0.7597335175224443.
The mean squared error is 0.2402664824775557.
The root mean squared error is 0.49016985064113816.
The standard deviation is 1.0000000000000007.


# Regression when we fill the null cells with the mean value

In [92]:
#cleaning data by removing all NaN
data_nonulls_df=All_data_df.dropna()
data_nonulls_df.describe()

,Year,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
count,412.000000,412.000000,412.000000,4.120000e+02,412.000000,412.000000,412.000000,412.000000
mean,2008.097087,28182.057263,77.276762,1.240653e+08,140.374029,1.770777,9.291748,33.317541
std,9.452933,15569.616894,4.937650,1.810933e+08,143.561538,0.403363,8.971921,17.053884
min,1981.000000,1136.755200,57.200000,1.681442e+07,2.200000,0.880000,1.800000,5.596642
25%,2001.000000,15389.205930,74.986000,5.132588e+07,25.175000,1.440000,3.700000,19.082791
50%,2009.500000,27373.057350,78.123000,6.647405e+07,98.750000,1.755000,5.400000,30.553422
75%,2016.000000,40024.395962,81.029000,1.276360e+08,233.450000,2.000000,11.075000,47.778956
max,2022.000000,76359.999520,84.910000,1.417173e+09,524.000000,3.050000,53.700000,69.851555


In [93]:
#,"G20 Value"
data_training2=All_data_df.drop(columns=["LOCATION","Year"])
data_training2=data_training2.fillna({'GDP Value':	28182.05,
                                      'LifeExp Value':77.27,
                                      'TotalFertilityRate(LiveBirthsPerWoman)':1.77,
                                      'Unemployment Value':6.70,
                                      'Population Value':1.240653e+08,
                                      'Education Value':33.31,
                                      'InfantMortalityRate(InfantDeathsPer1000LiveBirths)':9.29,
                                      'PopulationDensityAsOf1July(PersonsPerSquareKm)':140.37})
fertility_y = data_training2['TotalFertilityRate(LiveBirthsPerWoman)']
X=data_training2.drop(columns="TotalFertilityRate(LiveBirthsPerWoman)")
#X=data_training.copy()


model = LinearRegression()
model.fit(X, fertility_y)
coef=model.coef_
intercept=model.intercept_
predicted_y_values = model.predict(X)
predicted=data_training2.copy()
predicted["fertility predicted"]=predicted_y_values
predicted.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value,fertility predicted
0,2412.627589,70.610,10275000.0,1.3,3.45,20.1,33.31,2.240850
1,2383.188902,70.700,10508200.0,1.4,3.54,19.8,33.31,2.230553
2,2577.332834,70.790,10700500.0,1.4,3.43,20.3,33.31,2.251620
3,2752.620592,70.880,10906900.0,1.4,3.34,19.5,33.31,2.220261
4,2902.590472,70.872,11121600.0,1.4,3.15,18.7,33.31,2.187041


In [94]:
from sklearn.metrics import mean_squared_error, r2_score
score = model.score(X, fertility_y, sample_weight=None)
r2 = r2_score(fertility_y, predicted_y_values)
mse = mean_squared_error(fertility_y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(fertility_y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.75992002862865.
The r2 is 0.75992002862865.
The mean squared error is 0.3276542535926726.
The root mean squared error is 0.5724109132368744.
The standard deviation is 1.1682342640365762.


# Normalized Results with mean filling the null values

In [95]:

data_training3=data_training2.copy()
columns=[]
column_names=data_training3.columns.values.tolist()
columns.append(column_names)

data_training4 = StandardScaler().fit_transform(data_training3[column_names])
data_training4=pd.DataFrame(data_training4, columns=column_names,index=data_training3.index)

data_training4.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
0,-0.701967,-0.321901,-0.306640,-0.293999,0.965910,-0.068857,-0.000159
1,-0.703480,-0.309132,-0.305502,-0.293869,1.042950,-0.079929,-0.000159
2,-0.693503,-0.296362,-0.304563,-0.293869,0.948791,-0.061476,-0.000159
3,-0.684495,-0.283593,-0.303555,-0.293869,0.871751,-0.091000,-0.000159
4,-0.676788,-0.284728,-0.302507,-0.293869,0.709113,-0.120525,-0.000159


In [96]:
fertility_y = data_training4['TotalFertilityRate(LiveBirthsPerWoman)']
X=data_training4.drop(columns="TotalFertilityRate(LiveBirthsPerWoman)")
#X=data_training.copy()


model = LinearRegression()
model.fit(X, fertility_y)
coef=model.coef_
intercept=model.intercept_
predicted_y_values = model.predict(X)
predicted=data_training4.copy()
predicted["fertility predicted"]=predicted_y_values
predicted.head()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value,fertility predicted
0,-0.701967,-0.321901,-0.306640,-0.293999,0.965910,-0.068857,-0.000159,-0.069113
1,-0.703480,-0.309132,-0.305502,-0.293869,1.042950,-0.079929,-0.000159,-0.077928
2,-0.693503,-0.296362,-0.304563,-0.293869,0.948791,-0.061476,-0.000159,-0.059894
3,-0.684495,-0.283593,-0.303555,-0.293869,0.871751,-0.091000,-0.000159,-0.086738
4,-0.676788,-0.284728,-0.302507,-0.293869,0.709113,-0.120525,-0.000159,-0.115173


In [97]:
from sklearn.metrics import mean_squared_error, r2_score
score = model.score(X, fertility_y, sample_weight=None)
r2 = r2_score(fertility_y, predicted_y_values)
mse = mean_squared_error(fertility_y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(fertility_y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.75992002862865.
The r2 is 0.75992002862865.
The mean squared error is 0.2400799713713499.
The root mean squared error is 0.4899795621975981.
The standard deviation is 1.000000000000001.


In [104]:

data_training4[column_names].corr()

,GDP Value,LifeExp Value,Population Value,PopulationDensityAsOf1July(PersonsPerSquareKm),TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Education Value
GDP Value,1.000000,0.608964,-0.120477,0.116192,-0.369848,-0.429199,0.203807
LifeExp Value,0.608964,1.000000,-0.234118,0.124810,-0.678454,-0.851913,0.164429
Population Value,-0.120477,-0.234118,1.000000,-0.026170,0.107998,0.245666,-0.048674
PopulationDensityAsOf1July(PersonsPerSquareKm),0.116192,0.124810,-0.026170,1.000000,0.209669,0.049724,0.018065
TotalFertilityRate(LiveBirthsPerWoman),-0.369848,-0.678454,0.107998,0.209669,1.000000,0.847708,-0.061577
InfantMortalityRate(InfantDeathsPer1000LiveBirths),-0.429199,-0.851913,0.245666,0.049724,0.847708,1.000000,-0.074807
Education Value,0.203807,0.164429,-0.048674,0.018065,-0.061577,-0.074807,1.000000
